In [37]:
!pip install -U -q peft --progress-bar off
!pip install -U -q bitsandbytes --progress-bar off
!pip install -U -q transformers --progress-bar off

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [25]:
%ls "/content/drive/MyDrive/NLP_COURSE"

credentials.py  finetune.ipynb  finetuning_dataset.jsonl  Inference.ipynb  logs/  output/


In [38]:
%cd "/content/drive/MyDrive/NLP_COURSE"

/content/drive/MyDrive/NLP_COURSE


# Let's download the checkpoint
The saved checkpoint in the finetuning notebook is already uploaded to the Github repository!

In [27]:
!wget --no-check-certificate https://github.com/AliNajafi1998/VBYO2024/raw/main/outputs/ft_amazon.zip
!unzip ft_amazon.zip -d "ft_amazon"

--2024-07-13 19:48:02--  https://github.com/AliNajafi1998/VBYO2024/raw/main/outputs/ft_amazon.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/AliNajafi1998/VBYO2024/main/outputs/ft_amazon.zip [following]
--2024-07-13 19:48:03--  https://raw.githubusercontent.com/AliNajafi1998/VBYO2024/main/outputs/ft_amazon.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80099458 (76M) [application/zip]
Saving to: ‘ft_amazon.zip’

ft_amazon.zip       100%[===================>]  76.39M  70.2MB/s    in 1.1s    

2024-07-13 19:48:04 (70.2 MB/s) - ‘ft_amazon.zip’ saved [80099458/80099458]

Archive:  ft_amazo

In [31]:
%ls "/content/drive/MyDrive/NLP_COURSE"

credentials.py  finetuning_dataset.jsonl  ft_amazon.zip    logs/
finetune.ipynb  ft_amazon/                Inference.ipynb  output/


In [28]:
PROJECT_ROOT_PATH = "/content/drive/MyDrive/NLP_COURSE"
MODEL_DIR = f"{PROJECT_ROOT_PATH}/ft_amazon"

In [32]:
# @title Dependencies

import sys, os
from pprint import pprint
from pathlib import Path
from tqdm import tqdm, tqdm_pandas
tqdm.pandas()


import torch
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    PeftModel,
    )

from transformers import (
    BitsAndBytesConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
)

from credentials import HF_TOKEN
from huggingface_hub import login
login(HF_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [33]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"


# load the base model in 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True,
)
model.config.use_cache = False

# load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    use_fast=True,
    trust_remote_code=True,
    use_auth_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:778: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [34]:
#@title adding the LoRA adapters to the main model!
model = PeftModel.from_pretrained(
    model,
    MODEL_DIR,
    is_trainable=True,
    repo_type="file"
)
model = model.eval()

In [35]:
def get_text(question):
    text = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

    You are a helpful assistant for Amazon Shoe department. Your task is to help the customers find the best shoes of the world! <|eot_id|><|start_header_id|>user<|end_header_id|>

    {0} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""".format(question)
    return text


def generate_answer(model,
                    tokenizer,
                    question):

    text = get_text(question)
    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    generation_config = {
        "do_sample": True,
        "temperature": 0.6,
        "top_p": 0.9,
        "max_length":300,
        "num_return_sequences":1,
    }

    with torch.no_grad():
        outputs = model.generate(**inputs,**generation_config)

    decoded_output = tokenizer.decode(outputs[0])

    split_token = "<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
    decoded_output = decoded_output.split(split_token)[-1]
    return decoded_output.replace("<|eot_id|>", "")


In [36]:
output = generate_answer(model,
                         tokenizer,
                         "What are the best women shoes from Geox?")
pprint(output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


('\n'
 '\n'
 'Hello! Here are some women shoes from Geox that you might like:\n'
 "- Geox Women's D Rubidia a Sneaker (£29.00 - £115.51)\n"
 "- Geox Women's D Nebula a Sneaker (£35.00 - £111.52)\n"
 "- Geox Women's D Rubidia a Sneaker (£29.00 - £115.51)\n"
 "- Geox Women's D Nebula a Sneaker (£35.00 - £111.52)\n"
 "- Geox Women's D Kaula B Sneaker (£44.99 - £122.52)\n"
 "- Geox Women's D Rubidia a Sneaker (£29.00 - £115.51)\n"
 "- Geox Women's D Nebula a Sneaker (£35.00 - £111.52) ")
